# Explanation
This notebook utilizes the stevelib implementation of the Google Snap to Roads API (https://developers.google.com/maps/documentation/roads/snap).  It takes in a time-series lat/long dataframe and sends it to Google to get a dataframe back of snapped-to-roads lat and long values.  This function is wrapped in the `check()` function, which takes in:
- dataframe with a `lat` and `long` columns, and a `datetime` column of datetimes
- a datetime which represents the last point in the series to snap to
- (optional) increment (default, increment=1) which allows you to send every point, or only every other point (increment=2), if you want to simulate sparser data

In [1]:
import pandas as pd

import requests

from bokeh.models import WheelZoomTool
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_notebook

from stevelib import convert_gpx_to_df, snap_to_roads, plot2, transform_wgs84_to_web_mercator


output_notebook()

Loading BokehJS ...

In [2]:
# in this case we're getting our lat/long timeseries from a gpx file
df = convert_gpx_to_df('test_data/2019-08-27 151203.gpx')
df.head()

,lat,long
datetime,,
2019-08-27 20:37:56+00:00,37.787150,-122.403388
2019-08-27 20:37:59+00:00,37.787155,-122.403337
2019-08-27 20:38:00+00:00,37.787253,-122.403417
2019-08-27 20:38:01+00:00,37.787413,-122.403403
2019-08-27 20:38:02+00:00,37.787475,-122.403462


In [3]:
# TODO: add actual time spacingoutness?

In [4]:
# plot the end position after using the snap to roads algorithm
def check(df, timestamp, increment=1, ref_lat=None, ref_long=None):
    # plot this
    p = figure(
       x_axis_type="mercator", 
       y_axis_type="mercator",
       match_aspect=True,  # make sure the map isn't skewed?
       tools="pan,wheel_zoom,reset")
    p.add_tile(get_provider(Vendors.CARTODBPOSITRON))
    p.toolbar.active_scroll = p.select_one(WheelZoomTool) 
    
    a = df[:timestamp]  # take only the date from BEFORE the timestamp
    b = a.iloc[-80::increment]  # take just the last 80 of these, incremented by increment (not best way only gets 80)
    
    # original data
    plot2(p, b['lat'].values, b['long'].values, color='#d62728', size=5)

    output = snap_to_roads(b)  # call the snap to roads API and get a df with lat/longs snapped to the road
    
    # plot new route and the last point 
    x2, y2 = transform_wgs84_to_web_mercator(output['long'].values, output['lat'].values) 
    p.line(x2, y2, line_width=2, legend='Snap to road')
    p.x(x2[-1], y2[-1], size=25)
    
    
    # if exists plot the reference position too
    if ref_lat and ref_long:
        x3, y3 = transform_wgs84_to_web_mercator(ref_long, ref_lat) 
        p.x(x3, y3, color='#000000', size=50, legend='Reference position')
    
    show(p)

In [5]:
p = figure(
   x_axis_type="mercator", 
   y_axis_type="mercator",
   match_aspect=True,  # make sure the map isn't skewed?
   tools="pan,wheel_zoom,reset")
p.add_tile(get_provider(Vendors.CARTODBPOSITRON))
p.toolbar.active_scroll = p.select_one(WheelZoomTool) 

# original data
plot2(p, df['lat'].values, df['long'].values, color='#d62728', size=5)
# show(p)

In [6]:
# check(df.copy(), '2019-08-02 21:37:19+00:00')
check(df.copy(), '2019-08-27 13:42:00+00:00-07:00', ref_lat=37.787619, ref_long=-122.411597)
# DATA LOSS

In [7]:
# check(df.copy(), '2019-08-02 22:01:33+00:00')
check(df.copy(), '2019-08-27 13:56:24+00:00-07:00', ref_lat=37.789497, ref_long=-122.406154)

In [8]:
# check(df.copy(), '2019-08-02 22:19:08+00:00')
check(df.copy(), '2019-08-27 14:07:06+00:00-07:00', 1, ref_lat=37.789069, ref_long=-122.406288)

In [9]:
# check(df.copy(), '2019-08-02 22:41:09+00:00')
check(df.copy(), '2019-08-27 14:27:07+00:00-07:00', ref_lat=37.796102, ref_long=-122.398466)

In [10]:
check(df.copy(), '2019-08-27 14:38:48+00:00-07:00', ref_lat=37.79513, ref_long=-122.400149)

In [11]:
check(df.copy(), '2019-08-27 14:51:37+00:00-07:00', ref_lat=37.795705, ref_long=-122.401456)